# 🔒 FraudGuard Training Notebook

**AD-RL-GNN Fraud Detection** | Full training pipeline with mini-batch processing

This notebook trains the FraudGuard model on the IEEE-CIS fraud detection dataset using:
- **NeighborLoader** for memory-efficient mini-batch training
- **FAISS** for similarity graph construction (GPU if available, CPU fallback)
- **FocalLoss** for class-imbalanced learning

**Target Metrics:**
- Specificity: 98.72%
- G-Means Improvement: 18.11%
- P95 Latency: <100ms

## 1️⃣ Setup Environment

In [ ]:
# Mount Google Drive for data storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository
!git clone https://github.com/govind104/fraudguard.git
%cd fraudguard

In [ ]:
# Install dependencies
# Note: faiss-gpu may not be available on Python 3.12
# The code will fallback to faiss-cpu automatically
# GNN training STILL runs on GPU - only graph building uses CPU FAISS
!pip install -q torch torch-geometric pandas numpy scikit-learn pyyaml structlog

# Try faiss-gpu first, fallback to faiss-cpu
import subprocess
result = subprocess.run(['pip', 'install', '-q', 'faiss-gpu'], capture_output=True)
if result.returncode != 0:
    print('⚠️ faiss-gpu not available, using faiss-cpu')
    print('   (Graph building on CPU, but GNN training still runs on GPU!)')
    !pip install -q faiss-cpu
else:
    print('✓ faiss-gpu installed')

# Install repo in editable mode
!pip install -e .

print('\n✓ Environment setup complete')

## 2️⃣ Configuration

In [ ]:
import os

# ==============================================
# CONFIGURATION - UPDATE THESE PATHS AS NEEDED
# ==============================================

# Data paths - Point to your Google Drive folders
DATA_DIR = "/content/drive/MyDrive/ieee-fraud-detection"
MODELS_DIR = "/content/drive/MyDrive/fraudguard-models"
LOGS_DIR = "/content/drive/MyDrive/fraudguard-logs"

# Training parameters
SAMPLE_FRAC = 1.0      # Use full dataset (1.0 = 100%)
MAX_EPOCHS = 100
BATCH_SIZE = 4096      # Reduce to 2048 or 1024 if OOM
NUM_NEIGHBORS = [25, 10]  # 2-hop neighborhood sampling

# Create directories
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")
print(f"Logs: {LOGS_DIR}")
print(f"\nBatch size: {BATCH_SIZE}")
print(f"Sample fraction: {SAMPLE_FRAC*100:.0f}%")

## 3️⃣ Verify GPU and FAISS

In [ ]:
import torch
import faiss

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✓ GNN training will run on GPU")
else:
    print("\n⚠️ WARNING: No GPU detected. Go to Runtime > Change runtime type > GPU")

# Check FAISS GPU
faiss_gpus = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
print(f"\nFAISS GPUs: {faiss_gpus}")
if faiss_gpus == 0:
    print("   (Using CPU FAISS for graph building - this is OK)")

## 4️⃣ Load and Preprocess Data

In [ ]:
import sys
sys.path.insert(0, '/content/fraudguard')

from pathlib import Path
from src.data.loader import FraudDataLoader
from src.data.preprocessor import FeaturePreprocessor
from src.data.graph_builder import GraphBuilder
from src.utils.config import load_data_config
from src.utils.device_utils import set_seed, get_device

set_seed(42)
device = get_device()
print(f"Using device: {device}")

# Load config and override path with notebook variable
data_cfg = load_data_config()
data_cfg.paths.raw_data_dir = Path(DATA_DIR)

# Load data with corrected path
loader = FraudDataLoader(config=data_cfg)
df = loader.load_train_data(sample_frac=SAMPLE_FRAC)
train_df, val_df, test_df = loader.create_splits(df)

print(f"\nData loaded:")
print(f"  Train: {len(train_df):,}")
print(f"  Val: {len(val_df):,}")
print(f"  Test: {len(test_df):,}")
print(f"  Fraud rate: {df['isFraud'].mean()*100:.2f}%")

## 5️⃣ Build or Load Graph

In [ ]:
import torch
import os
import gc
from src.utils.config import load_model_config

GRAPH_CACHE = f"{MODELS_DIR}/edges_full.pt"
FEATURES_CACHE = f"{MODELS_DIR}/features_full.pt"

# 1. Aggressive Cleanup before starting
# We don't need the raw pandas DataFrames anymore if we have the tensors
# This frees up ~1-2GB of System RAM immediately
del df
gc.collect()

# 2. Preprocess features - KEEP ON CPU INITIALLY
# We do NOT move to .to(device) yet to save GPU VRAM for FAISS
print("Preprocessing features (CPU)...")
prep = FeaturePreprocessor()
X_train = prep.fit_transform(train_df)
X_val = prep.transform(val_df)
X_test = prep.transform(test_df)

# In Cell 6, the labels were already extracted before the del
# If you get a NameError for train_df, save labels BEFORE the del in Cell 5:

# Prepare labels
train_labels = torch.tensor(train_df["isFraud"].values, dtype=torch.long)
val_labels = torch.tensor(val_df["isFraud"].values, dtype=torch.long)
test_labels = torch.tensor(test_df["isFraud"].values, dtype=torch.long)

# Free up dataframe memory
del train_df, val_df, test_df
gc.collect()

# Concatenate on CPU
X_full = torch.cat([X_train, X_val, X_test])
print(f"Features shape: {X_full.shape}")

# 3. Graph Logic
if os.path.exists(GRAPH_CACHE):
    print("Loading cached graph from Google Drive...")
    edge_index = torch.load(GRAPH_CACHE)
    print(f"Loaded {edge_index.shape[1]:,} edges")
else:
    print("Building graph with STRICTER threshold (0.9)...")
    
    # LOAD & OVERRIDE CONFIG
    # We force the threshold to 0.9 to reduce edge count by ~70-80%
    model_cfg = load_model_config()
    model_cfg.graph.similarity_threshold = 0.9 
    
    builder = GraphBuilder(config=model_cfg)
    
    # Fit on Train
    print("Fitting FAISS index...")
    train_edges = builder.fit(X_train)
    
    # Transform Remainder (Val/Test)
    # We pass the config explicitly if your class supports it, 
    # otherwise the builder.config update above handles it
    print("Searching for neighbors (this is the memory-intensive part)...")
    edge_index = builder.transform(torch.cat([X_val, X_test]), train_size=len(X_train))
    
    # Verify leak-free
    print("Verifying graph integrity...")
    builder.verify_no_leakage(edge_index, train_size=len(X_train))
    
    # Cache to Drive
    print(f"Saving {edge_index.shape[1]:,} edges to Drive...")
    torch.save(edge_index, GRAPH_CACHE)
    torch.save(X_full, FEATURES_CACHE)
    print(f"Graph cached to {GRAPH_CACHE}")

# 4. Final Move to GPU
# Now that the heavy lifting is done, we move data to GPU
print("Moving data to GPU...")
X_full = X_full.to(device)
edge_index = edge_index.to(device)

print(f"\nFinal Graph: {edge_index.shape[1]:,} edges on {device}")

## 6️⃣ Setup Mini-Batch Training

In [ ]:
from torch_geometric.loader import NeighborLoader
from torch_geometric.data import Data
import torch.nn.functional as F

# Prepare labels
train_labels = torch.tensor(train_df["isFraud"].values, dtype=torch.long)
val_labels = torch.tensor(val_df["isFraud"].values, dtype=torch.long)
test_labels = torch.tensor(test_df["isFraud"].values, dtype=torch.long)
all_labels = torch.cat([train_labels, val_labels, test_labels]).to(device)

# Create masks
n = len(X_full)
train_mask = torch.zeros(n, dtype=torch.bool)
val_mask = torch.zeros(n, dtype=torch.bool)
test_mask = torch.zeros(n, dtype=torch.bool)
train_mask[:len(X_train)] = True
val_mask[len(X_train):len(X_train)+len(X_val)] = True
test_mask[len(X_train)+len(X_val):] = True

# Create PyG Data object
data = Data(x=X_full, edge_index=edge_index, y=all_labels)
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

# Create NeighborLoader for mini-batch training
# This is the KEY to avoiding OOM - samples subgraphs instead of full graph
train_loader = NeighborLoader(
    data,
    num_neighbors=NUM_NEIGHBORS,  # [25, 10] = 25 first-hop, 10 second-hop
    batch_size=BATCH_SIZE,
    input_nodes=train_mask,
    shuffle=True,
)

val_loader = NeighborLoader(
    data,
    num_neighbors=NUM_NEIGHBORS,
    batch_size=BATCH_SIZE,
    input_nodes=val_mask,
    shuffle=False,
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Neighbor sampling: {NUM_NEIGHBORS}")

## 7️⃣ Train Model

In [ ]:
from src.models import FraudGNN, FocalLoss, compute_class_weights
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
import time

# Initialize model
model = FraudGNN(in_channels=X_full.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
weights = compute_class_weights(train_labels, device)
criterion = FocalLoss(alpha=0.5, gamma=4, weight=weights)

# Training config
best_gmeans = 0
patience = 30
patience_counter = 0
history = []

print("Starting training...\n")
print(f"{'Epoch':>5} | {'Loss':>8} | {'Spec':>8} | {'Recall':>8} | {'F1':>8} | {'G-Means':>8}")
print("-" * 60)

start_time = time.time()

for epoch in range(MAX_EPOCHS):
    # Training
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out[:batch.batch_size], batch.y[:batch.batch_size])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    
    # Validation every 5 epochs
    if epoch % 5 == 0:
        model.eval()
        all_preds, all_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                out = model(batch.x, batch.edge_index)
                pred = out[:batch.batch_size].argmax(dim=1)
                all_preds.extend(pred.cpu().numpy())
                all_true.extend(batch.y[:batch.batch_size].cpu().numpy())
        
        cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        gmeans = np.sqrt(tpr * tnr)
        f1 = f1_score(all_true, all_preds, zero_division=0)
        
        history.append({'epoch': epoch, 'loss': avg_loss, 'spec': tnr, 'recall': tpr, 'f1': f1, 'gmeans': gmeans})
        
        print(f"{epoch+1:>5} | {avg_loss:>8.4f} | {tnr*100:>7.2f}% | {tpr*100:>7.2f}% | {f1*100:>7.2f}% | {gmeans*100:>7.2f}%")
        
        # Early stopping
        if gmeans > best_gmeans:
            best_gmeans = gmeans
            patience_counter = 0
            torch.save(model.state_dict(), f"{MODELS_DIR}/best_model.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience // 5:
                print(f"\nEarly stopping at epoch {epoch+1}")
                break

train_time = time.time() - start_time
print(f"\nTraining complete in {train_time/60:.1f} minutes")
print(f"Best validation G-Means: {best_gmeans*100:.2f}%")

## 8️⃣ Evaluate on Test Set

In [ ]:
import time

# Load best model
model.load_state_dict(torch.load(f"{MODELS_DIR}/best_model.pt"))
model.eval()

# Full neighborhood for test evaluation
test_loader = NeighborLoader(
    data,
    num_neighbors=[-1, -1],  # Full neighborhood
    batch_size=BATCH_SIZE,
    input_nodes=test_mask,
    shuffle=False,
)

all_preds, all_true = [], []
latencies = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        start = time.perf_counter()
        out = model(batch.x, batch.edge_index)
        latencies.append((time.perf_counter() - start) * 1000)
        pred = out[:batch.batch_size].argmax(dim=1)
        all_preds.extend(pred.cpu().numpy())
        all_true.extend(batch.y[:batch.batch_size].cpu().numpy())

# Compute metrics
cm = confusion_matrix(all_true, all_preds, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
gmeans = np.sqrt(tpr * tnr)
f1 = f1_score(all_true, all_preds, zero_division=0)

print("=" * 60)
print("FINAL TEST RESULTS")
print("=" * 60)
print(f"\nConfusion Matrix:")
print(f"  TP: {tp:,}  |  FN: {fn:,}")
print(f"  FP: {fp:,}  |  TN: {tn:,}")
print(f"\nPerformance:")
print(f"  Specificity:  {tnr*100:.2f}%  (CV target: 98.72%)")
print(f"  Recall:       {tpr*100:.2f}%")
print(f"  F1 Score:     {f1*100:.2f}%")
print(f"  G-Means:      {gmeans*100:.2f}%")
print(f"\nLatency:")
print(f"  Mean: {np.mean(latencies):.1f}ms")
print(f"  P95:  {np.percentile(latencies, 95):.1f}ms  (CV target: <100ms)")
print(f"  P99:  {np.percentile(latencies, 99):.1f}ms")

## 9️⃣ Save Final Model

In [ ]:
# Save final model with metrics
torch.save({
    "model_state_dict": model.state_dict(),
    "config": {
        "in_channels": X_full.shape[1],
        "specificity": tnr,
        "recall": tpr,
        "gmeans": gmeans,
        "f1": f1,
    },
    "history": history,
}, f"{MODELS_DIR}/fraudguard_final.pt")

print(f"✓ Model saved to {MODELS_DIR}/fraudguard_final.pt")
print(f"✓ Best model saved to {MODELS_DIR}/best_model.pt")

## 🔟 CV Claims Comparison

In [ ]:
# CV Claims comparison
CV_CLAIMS = {
    "specificity": 98.72,
    "gmeans_improvement": 18.11,
    "p95_latency_ms": 100,
}

achieved_spec = tnr * 100
p95_latency = np.percentile(latencies, 95)

print("=" * 60)
print("CV CLAIMS COMPARISON")
print("=" * 60)
print(f"| {'Metric':<20} | {'Achieved':>12} | {'CV Claim':>12} | {'Status':>6} |")
print(f"|{'-'*22}|{'-'*14}|{'-'*14}|{'-'*8}|")

# Specificity
status_spec = "✓" if abs(achieved_spec - CV_CLAIMS['specificity']) <= 3 else "✗"
print(f"| {'Specificity':<20} | {achieved_spec:>11.2f}% | {CV_CLAIMS['specificity']:>11.2f}% | {status_spec:>6} |")

# Latency
status_lat = "✓" if p95_latency < CV_CLAIMS['p95_latency_ms'] else "✗"
print(f"| {'P95 Latency':<20} | {p95_latency:>10.1f}ms | {'<100':>10}ms | {status_lat:>6} |")

print("=" * 60)
print("\n✓ = PASS (within tolerance) | ✗ = INVESTIGATE")